In [5]:
from itertools import product

import pandas as pd
import pickle
class AssociationRecommendationModel:
    def __init__(self, data, rules):
        self.data = data
        self.rules = rules

    def get_top_associated_categories(self, item_ids, n=3):
        top_associated_categories = set()

        for item_id in item_ids:
            # Retrieve item information
            item_info = self.data[self.data['id'] == item_id]
            if item_info.empty:
                print(f"Item {item_id} not found in dataset. Skipping.")
                continue

            # Construct item category combination
            item_category = f"{item_info.iloc[0]['main_category']} - {item_info.iloc[0]['sub_category']}"

            # Filter rules to find relevant antecedents
            relevant_rules = self.rules[self.rules['antecedents'].apply(lambda x: item_category in x)]

            # Sort rules by lift and retrieve top categories
            relevant_rules = relevant_rules.sort_values(by='lift', ascending=False).head(n)

            # Add consequents to top_associated_categories
            for _, row in relevant_rules.iterrows():
                # Extract items from the frozenset
                top_associated_categories.update(list(row['consequents']))

        return list(top_associated_categories)


    def get_top_rated_items(self, associated_categories, top_n=2):
        top_items = []
        for category in associated_categories:
            # Split the main and subcategories
            main_category, sub_category = category.split(" - ")
            # Filter items within the given category
            category_items = self.data[
                (self.data['main_category'] == main_category) &
                (self.data['sub_category'] == sub_category)
            ]
            # Get top items based on ratings
            top_rated_items = category_items.sort_values(by='ratings', ascending=False).head(top_n)
            top_items.extend(top_rated_items['id'].tolist())
        return top_items

    def recommend(self, item_ids, associated_n=5, top_n=2):
        # Get top associated categories for the input items
        top_associated_categories = self.get_top_associated_categories(item_ids, n=associated_n)
        print(f"Top associated categories: {top_associated_categories}")
        # Get highest-rated items from the associated categories
        top_rated_items = self.get_top_rated_items(top_associated_categories, top_n=top_n)
        return top_rated_items

    def save(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump(self, f)
        print(f"Model saved to {file_path}")

    @staticmethod
    def load(file_path):
        with open(file_path, 'rb') as f:
            model = pickle.load(f)
        print(f"Model loaded from {file_path}")
        return model


In [6]:
if __name__ == "__main__":
    # Load datasets
    product = pd.read_csv('test_data.csv')
    rules = pd.read_csv('association_rules.csv')  # Ensure this file contains 'antecedents', 'consequents', and 'lift'

    # Initialize the model
    model = AssociationRecommendationModel(data=product, rules=rules)

    # Save the model to a Pickle file
    model.save('D:/RecommendModel/Saved_model/association_recommendation_model.pkl')

    loaded_model = AssociationRecommendationModel.load('D:/RecommendModel/Saved_model/association_recommendation_model.pkl')

    example_item_ids = ['tvc119790', 'mcl257235']
    recommendations = loaded_model.recommend(item_ids=example_item_ids, associated_n=5, top_n=10)
    print("Final Recommendations:", recommendations)

Model saved to D:/RecommendModel/Saved_model/association_recommendation_model.pkl
Model loaded from D:/RecommendModel/Saved_model/association_recommendation_model.pkl
Top associated categories: ['w', ')', 'o', 'g', 'c', 'r', ' ', '}', 'J', 'n', 'l', 'i', '(', 'E', 'W', 't', '-', 's', 'e', 'P', '"', 'T', 'f', 'h', 'I', '&', "'", 'a', '{', 'm', 'z']


ValueError: not enough values to unpack (expected 2, got 1)